# Ansbile playbook

## 1. Simple playbook

### 1.1. Ping test

See [ping.yml](./ping.yml)

#### 1.1.1. Ping remote server as current user

In [ ]:
ansible-playbook ping.yml --tags "G1"

#### 1.1.2. Ping remote server as become user

- `@password.yml`: The external arguments file, encrypted
- `--vault-id`: The password to decrypted `@password.yml` file

In [ ]:
ansible-playbook ping.yml --tags "G2" --vault-id vault-id -e "@password.yml"

### 1.2. Ansible variables

See [vars.yml](./vars.yml)

#### 1.2.1. Ansible builtin variables

In [ ]:
ansible-playbook vars.yml --tags "G1"

#### 1.2.2. User define variables

- Without external args

In [ ]:
ansible-playbook vars.yml --tags "G2"

- With external args
    - `-e`: external arguments, if will override the same argument defined in playbook

In [ ]:
ansible-playbook vars.yml -e "{ user_name: 'Emma', user_age: 32 }" --tags "G2"

#### 1.2.3. Include variables file

In [ ]:
ansible-playbook vars.yml --tags "G3"

#### 1.2.4. Environment variables

In [ ]:
ENV_VAR="HELLO WORLD" ansible-playbook vars.yml --tags "G4"

#### 1.2.5. Show ansible variables and facts variables

In [ ]:
ansible-playbook vars.yml --tags "G5"

#### 1.2.6. Set variable by task

- `set_fact`: `set_fact` module can set variable in task

In [ ]:
ansible-playbook vars.yml --tags "G6"

### 1.3. Conditionals

See [Conditional](./conditional.yml)

#### 1.3.1. Basic conditionals with `when`

- `when <conditional>`: Task execute when value of conditional statement is `True`

In [ ]:
ansible-playbook conditional.yml --tags "G1"

#### 1.3.2. Conditionals based on `ansible_facts`

In [ ]:
ansible-playbook conditional.yml --tags "G2"

#### 1.3.3. Conditions based on registered variables

- `register`: Register a variable to save result of task
  ```yaml
  - name: "Demo task"
    register: "result"
    ...
  ```

In [ ]:
ansible-playbook conditional.yml --tags "G3"

#### 1.3.4. Conditions based on operation result

- `skipped`: Registered variable is marked as 'skipped', the task was skipped
  ```yaml
  when: "result is skipped"  # result is register variable
  ```

In [ ]:
ansible-playbook conditional.yml --tags "G4"

- `succeeded`: Registered variable is marked as 'succeeded', the task worked succsessful
  ```yaml
  when: "result is succeeded"  # result is register variable
  ```

In [ ]:
CMD="ls -al" ansible-playbook conditional.yml --tags "G4"

- `failed`: Registered variable is marked as 'failed', the task finish with error

In [ ]:
CMD="bad_command" ansible-playbook conditional.yml --tags "G4"

### 1.4. Loop

See [loop.yml](./loop.yml)

#### 1.4.1. Simple loop

- `with_items <collection>`

In [ ]:
ansible-playbook loop.yml --tags "G1"

#### 1.4.2. Nested loop

- `with_nested: [<collection1>, <collection2>, ...]`

In [ ]:
ansible-playbook loop.yml --tags "G2"

#### 1.4.3. Loop with task results

- `with_items <register variable>`

In [ ]:
ansible-playbook loop.yml --tags "G3"

#### 1.4.4. Loop in dictionary

In [ ]:
ansible-playbook loop.yml --tags "G4"

#### 1.4.5. Loop in dictionary

In [ ]:
ansible-playbook loop.yml --tags "G5"

### 1.5. Block

See [block.yml](./block.yml)

#### 1.5.1. Run task group by blocks

In [ ]:
ansible-playbook block.yml --tags "G1"

#### 1.5.2. Error handle by blocks

In [ ]:
ansible-playbook block.yml --tags "G2"